In [1]:
import requests
import pandas as pd

In [4]:
resp = requests.get("http://www.datiopen.it/export/csv/Mappa-dei-telefoni-pubblici-in-Italia.csv")
if resp.ok:
    data = resp.text
    f = open("input.csv", "w")
    f.write(data)
    f.close()

In [55]:
df = pd.read_csv("input.csv", sep=";")
df

,Comune,Provincia,Regione,Nome,Anno inserimento,Data e ora inserimento,Identificatore in OpenStreetMap,Longitudine,Latitudine
0,ALTRO,ALTRO,ALTRO,NaN,2010,2010-04-04T20:09:21Z,684571448,13.715068,46.497074
1,ALTRO,ALTRO,ALTRO,NaN,2010,2010-06-28T15:26:18Z,794445837,10.508641,46.875588
2,ALTRO,ALTRO,ALTRO,NaN,2010,2010-07-20T20:56:09Z,826760810,9.017316,45.998330
3,ALTRO,ALTRO,ALTRO,NaN,2010,2010-09-15T20:35:02Z,913945114,13.903188,45.621851
4,ALTRO,ALTRO,ALTRO,NaN,2011,2011-04-16T23:30:32Z,1247809327,9.538167,46.341037
...,...,...,...,...,...,...,...,...,...
3818,BUSNAGO,MONZA-BRIANZA,Lombardia,NaN,2015,2015-11-24T09:36:31Z,3856107275,9.465435,45.616109
3819,FERMO,FERMO,Marche,NaN,2012,2012-03-26T06:25:14Z,615074344,13.718606,43.160290
3820,ANDRIA,BARLETTA-ANDRIA-TRANI,Puglia,NaN,2009,2009-07-14T07:30:29Z,395420004,16.304417,41.231526
3821,CANOSA DI PUGLIA,BARLETTA-ANDRIA-TRANI,Puglia,NaN,2016,2016-02-09T09:03:48Z,3996898138,16.065573,41.222895


In [39]:
regioni = df["Regione"].unique()
for reg in regioni:
    dfregione = df[df["Regione"] == reg]
    dfprovincia = dfregione.groupby("Provincia")["Provincia"].count()
    plot = dfprovincia.plot(kind="pie", y="Provincia")
    fig = plot.get_figure()
    fig.savefig("itphones_outputs/" + reg + ".png")
    fig.clear()

<Figure size 640x480 with 0 Axes>

In [40]:
pip install folium

  Using cached folium-0.19.5-py2.py3-none-any.whl (110 kB)
Note: you may need to restart the kernel to use updated packages.


In [41]:
import folium
dir(folium)

['Choropleth',
 'Circle',
 'CircleMarker',
 'ClickForLatLng',
 'ClickForMarker',
 'ColorLine',
 'ColorMap',
 'CssLink',
 'CustomIcon',
 'Div',
 'DivIcon',
 'Element',
 'FeatureGroup',
 'Figure',
 'FitBounds',
 'FitOverlays',
 'GeoJson',
 'GeoJsonPopup',
 'GeoJsonTooltip',
 'Html',
 'IFrame',
 'Icon',
 'JavascriptLink',
 'JsCode',
 'LatLngPopup',
 'LayerControl',
 'LinearColormap',
 'Link',
 'MacroElement',
 'Map',
 'Marker',
 'PolyLine',
 'Polygon',
 'Popup',
 'Rectangle',
 'RegularPolygonMarker',
 'StepColormap',
 'TileLayer',
 'Tooltip',
 'TopoJson',
 'Vega',
 'VegaLite',
 'WmsTileLayer',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_version',
 'branca',
 'elements',
 'features',
 'folium',
 'map',
 'raster_layers',
 'template',
 'utilities',
 'vector_layers']

In [70]:
from datetime import datetime
timeoffset = -8
center = (41.902782, 12.496366)
bounds = [[42.62115209225544, 15.669698244577547], [39.04096044837196, 19.97861416589663]]
colors = ["darkgreen","green","orange","red"]
locations = list(zip(df["Latitudine"], df["Longitudine"]))
m = folium.Map(center, zoom_start=7)
m.fit_bounds(bounds)
for i in range(len(locations)):
    loc = locations[i]
    isinside = (loc[0] <= bounds[0][0] and loc[0] >= bounds[1][0]) and (loc[1] >= bounds[0][1] and loc[1] <= bounds[1][1])
    if isinside:
        insdate = int(df["Anno inserimento"][i])
        agecolorindex = min(3, (datetime.now().year + timeoffset - insdate) // 2)
        color = colors[agecolorindex]
        icon = folium.Icon(color=color, fill_color=color)
        folium.Marker([loc[0], loc[1]], icon=icon).add_to(m)
m.save("itphones_outputs/index.html")

In [60]:
import date


5